In [13]:
import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Constants
API_KEY = '2qBcnThLnlearCE77ZchoF2uJLQ'  # Replace with your actual API key
SINCE_DATE = int(datetime(2014, 1, 1).timestamp())  # Jan 1, 2015
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/market/marketcap_realized_usd',
    'https://api.glassnode.com/v1/metrics/supply/lth_sum',
    'https://api.glassnode.com/v1/metrics/supply/sth_sum',
    'https://api.glassnode.com/v1/metrics/indicators/rhodl_ratio'
]

# Dictionary for stacked metrics: URL -> key to extract
METRICS_STACKED = {
    'https://api.glassnode.com/v1/metrics/supply/rcap_hodl_waves': ['24h', '1d_1w','1w_1m','1m_3m']  # Now accepts a list of keys
}

def fetch_glassnode_data(url, asset='BTC', key_to_extract=None):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'c': 'USD'
    }
    
    metric_base_name = url.split('/')[-1]
    
    # Set format based on whether it's a stacked metric
    if url in METRICS_STACKED:
        params['f'] = 'JSON'
    else:
        params['f'] = 'CSV'

    response = requests.get(url, params=params)
    if response.status_code == 200:
        # Handle stacked metric response
        if url in METRICS_STACKED and key_to_extract:
            data = response.json()
            # Create column name combining metric name and the specific key
            column_name = f"{metric_base_name}_{key_to_extract}"
            
            # Extract timestamps and the specific key we want
            timestamps = [entry['t'] for entry in data]
            values = [entry['o'][key_to_extract] for entry in data]
            
            df = pd.DataFrame({
                't': timestamps,
                column_name: values
            })
        else:
            # Handle regular metric response
            df = pd.read_csv(io.StringIO(response.text))
            # Keep original column names from CSV
            if len(df.columns) != 2:
                print(f"Unexpected number of columns in response from {url}: {df.columns}")
                return None
            df.columns = ['t', metric_base_name]
        
        df['t'] = pd.to_datetime(df['t'], unit='s')
        # Convert the metric column to numeric, excluding the timestamp column
        for col in df.columns:
            if col != 't':
                df[col] = pd.to_numeric(df[col], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
print("Fetching price data...")
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]

# Fetch regular metrics
print("Fetching regular metrics...")
for metric_url in METRICS:
    print(f"Fetching {metric_url.split('/')[-1]}...")
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

# Fetch stacked metrics
print("Fetching stacked metrics...")
for metric_url, keys in METRICS_STACKED.items():
    # Handle both single key (string) and multiple keys (list) cases
    if isinstance(keys, str):
        keys = [keys]  # Convert single key to list
    for key in keys:
        print(f"Fetching {metric_url.split('/')[-1]} - {key}...")
        metric_df = fetch_glassnode_data(metric_url, key_to_extract=key)
        if metric_df is not None:
            all_dfs.append(metric_df)

print("Merging all dataframes...")
merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

print("Final columns in merged_df:", merged_df.columns.tolist())

Fetching price data...
Fetching regular metrics...
Fetching marketcap_realized_usd...
Fetching lth_sum...
Fetching sth_sum...
Fetching rhodl_ratio...
Fetching stacked metrics...
Fetching rcap_hodl_waves - 24h...
Fetching rcap_hodl_waves - 1d_1w...
Fetching rcap_hodl_waves - 1w_1m...
Fetching rcap_hodl_waves - 1m_3m...
Merging all dataframes...
Final columns in merged_df: ['price_usd_close', 'marketcap_realized_usd', 'lth_sum', 'sth_sum', 'rhodl_ratio', 'rcap_hodl_waves_24h', 'rcap_hodl_waves_1d_1w', 'rcap_hodl_waves_1w_1m', 'rcap_hodl_waves_1m_3m']


In [14]:
def calculate_smoothed_bands_v3(df, column = 'marketcap_realized_usd', smoothing_fun='sma', smoothing_window=7, range_method='maxmin', range_window=90):
  # Calculate product of the two columns
  product = df[column].pct_change(periods=30) * 100
  # Calculate smoothed version based on method using the product
  if smoothing_fun == 'sma':
      smoothed_raw = product.rolling(window=smoothing_window).mean()
  elif smoothing_fun == 'ema':
      smoothed_raw = product.ewm(span=smoothing_window).mean()
  elif smoothing_fun == 'med':
      smoothed_raw = product.rolling(window=smoothing_window).median()
  else:
      raise ValueError("Invalid smoothing method. Choose 'sma', 'ema', or 'med'.")
  
  # Calculate bands based on smoothed data
  if range_method == 'maxmin':
      upper_band = smoothed_raw.rolling(window=range_window).max()
      lower_band = smoothed_raw.rolling(window=range_window).min()
  
  elif range_method == 'zscore':
      mean = smoothed_raw.rolling(window=range_window).mean()
      std = smoothed_raw.rolling(window=range_window).std()
      upper_band = mean + std
      lower_band = mean - std
  
  elif range_method == 'percentile':
      upper_band = smoothed_raw.rolling(window=range_window).quantile(0.95)
      lower_band = smoothed_raw.rolling(window=range_window).quantile(0.05)
  
  elif range_method == 'meandev':
      mean = smoothed_raw.rolling(window=range_window).mean()
      mean_deviation = (smoothed_raw - mean).abs().rolling(window=range_window).mean()
      upper_band = mean + mean_deviation
      lower_band = mean - mean_deviation
  
  else:
      raise ValueError("Invalid range method. Choose 'maxmin', 'zscore', 'percentile', or 'meandev'.")
  
  return smoothed_raw, upper_band, lower_band

In [15]:
def calculate_smoothed_bands_v4(df, columns=["sth_sum", "lth_sum"], smoothing_fun='sma', smoothing_window=7, range_method='maxmin', range_window=90):
  # Calculate product of the two columns
  product = df[columns[0]]/ df[columns[1]]
  # Calculate smoothed version based on method using the product
  if smoothing_fun == 'sma':
      smoothed_raw = product.rolling(window=smoothing_window).mean()
  elif smoothing_fun == 'ema':
      smoothed_raw = product.ewm(span=smoothing_window).mean()
  elif smoothing_fun == 'med':
      smoothed_raw = product.rolling(window=smoothing_window).median()
  else:
      raise ValueError("Invalid smoothing method. Choose 'sma', 'ema', or 'med'.")
  
  # Calculate bands based on smoothed data
  if range_method == 'maxmin':
      upper_band = smoothed_raw.rolling(window=range_window).max()
      lower_band = smoothed_raw.rolling(window=range_window).min()
  
  elif range_method == 'zscore':
      mean = smoothed_raw.rolling(window=range_window).mean()
      std = smoothed_raw.rolling(window=range_window).std()
      upper_band = mean + std
      lower_band = mean - std
  
  elif range_method == 'percentile':
      upper_band = smoothed_raw.rolling(window=range_window).quantile(0.95)
      lower_band = smoothed_raw.rolling(window=range_window).quantile(0.05)
  
  elif range_method == 'meandev':
      mean = smoothed_raw.rolling(window=range_window).mean()
      mean_deviation = (smoothed_raw - mean).abs().rolling(window=range_window).mean()
      upper_band = mean + mean_deviation
      lower_band = mean - mean_deviation
  
  else:
      raise ValueError("Invalid range method. Choose 'maxmin', 'zscore', 'percentile', or 'meandev'.")
  
  return smoothed_raw, upper_band, lower_band

In [16]:
def calculate_smoothed_bands_v5(df, column = 'rhodl_ratio', smoothing_fun='sma', smoothing_window=7, range_method='maxmin', range_window=90):
  # Calculate product of the two columns
  product = np.log(df[column])
  # Calculate smoothed version based on method using the product
  if smoothing_fun == 'sma':
      smoothed_raw = product.rolling(window=smoothing_window).mean()
  elif smoothing_fun == 'ema':
      smoothed_raw = product.ewm(span=smoothing_window).mean()
  elif smoothing_fun == 'med':
      smoothed_raw = product.rolling(window=smoothing_window).median()
  else:
      raise ValueError("Invalid smoothing method. Choose 'sma', 'ema', or 'med'.")
  
  # Calculate bands based on smoothed data
  if range_method == 'maxmin':
      upper_band = smoothed_raw.rolling(window=range_window).max()
      lower_band = smoothed_raw.rolling(window=range_window).min()
  
  elif range_method == 'zscore':
      mean = smoothed_raw.rolling(window=range_window).mean()
      std = smoothed_raw.rolling(window=range_window).std()
      upper_band = mean + std
      lower_band = mean - std
  
  elif range_method == 'percentile':
      upper_band = smoothed_raw.rolling(window=range_window).quantile(0.95)
      lower_band = smoothed_raw.rolling(window=range_window).quantile(0.05)
  
  elif range_method == 'meandev':
      mean = smoothed_raw.rolling(window=range_window).mean()
      mean_deviation = (smoothed_raw - mean).abs().rolling(window=range_window).mean()
      upper_band = mean + mean_deviation
      lower_band = mean - mean_deviation
  
  else:
      raise ValueError("Invalid range method. Choose 'maxmin', 'zscore', 'percentile', or 'meandev'.")
  
  return smoothed_raw, upper_band, lower_band

In [17]:
def calculate_smoothed_bands_v6(df, column=['rcap_hodl_waves_24h', 'rcap_hodl_waves_1d_1w', 'rcap_hodl_waves_1w_1m', 'rcap_hodl_waves_1m_3m'], smoothing_fun='sma', smoothing_window=7, range_method='maxmin', range_window=90):
    # Calculate product of the two columns
    product = df[column].sum(axis=1)
    # Calculate smoothed version based on method using the product
    if smoothing_fun == 'sma':
        smoothed_raw = product.rolling(window=smoothing_window).mean()
    elif smoothing_fun == 'ema':
        smoothed_raw = product.ewm(span=smoothing_window).mean()
    elif smoothing_fun == 'med':
        smoothed_raw = product.rolling(window=smoothing_window).median()
    else:
        raise ValueError("Invalid smoothing method. Choose 'sma', 'ema', or 'med'.")

    # Calculate bands based on smoothed data
    if range_method == 'maxmin':
        upper_band = smoothed_raw.rolling(window=range_window).max()
        lower_band = smoothed_raw.rolling(window=range_window).min()

    elif range_method == 'zscore':
        mean = smoothed_raw.rolling(window=range_window).mean()
        std = smoothed_raw.rolling(window=range_window).std()
        upper_band = mean + std
        lower_band = mean - std

    elif range_method == 'percentile':
        upper_band = smoothed_raw.rolling(window=range_window).quantile(0.95)
        lower_band = smoothed_raw.rolling(window=range_window).quantile(0.05)

    elif range_method == 'meandev':
        mean = smoothed_raw.rolling(window=range_window).mean()
        mean_deviation = (smoothed_raw - mean).abs().rolling(window=range_window).mean()
        upper_band = mean + mean_deviation
        lower_band = mean - mean_deviation

    else:
        raise ValueError("Invalid range method. Choose 'maxmin', 'zscore', 'percentile', or 'meandev'.")

    return smoothed_raw, upper_band, lower_band

In [18]:
merged_df['rcap_sma_zscore'], merged_df['rcap_upper'], merged_df['rcap_lower']= calculate_smoothed_bands_v3(merged_df, column='marketcap_realized_usd',smoothing_fun='sma', smoothing_window=7, range_method='zscore', range_window=180)
merged_df['sthlth_sma_meandev'], merged_df['sthlth_upper'], merged_df['sthlth_lower']= calculate_smoothed_bands_v4(merged_df, columns=["sth_sum", "lth_sum"],smoothing_fun='sma', smoothing_window=7, range_method='meandev', range_window=180)
merged_df['rhodl_sma_maxmin'], merged_df['rhodl_upper'], merged_df['rhodl_lower']= calculate_smoothed_bands_v5(merged_df, column = 'rhodl_ratio', smoothing_fun='sma', smoothing_window=7, range_method='maxmin', range_window=90)
merged_df['rhodlwave_sma_zscore'], merged_df['rhodlwave_upper'], merged_df['rhodlwave_lower']= calculate_smoothed_bands_v6(merged_df, column=['rcap_hodl_waves_24h', 'rcap_hodl_waves_1d_1w', 'rcap_hodl_waves_1w_1m', 'rcap_hodl_waves_1m_3m'], smoothing_fun='sma', smoothing_window=7, range_method='zscore', range_window=180)

In [19]:
import pandas as pd
from plotly.subplots import make_subplots

def create_indicator_chart(merged_df, indicator_column, chart_title, bands="on", upper_lower=["fee_upper", "fee_lower"]):
   # Filter data for the last year
   one_year_ago = datetime.now() - timedelta(days=730)
   merged_df_last_year = merged_df[merged_df.index > one_year_ago]

   # Define colors
   GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey
   BLUE_COLOR = 'rgb(0, 0, 255)'  # Blue
   GREEN_COLOR = 'rgb(0, 255, 0)'  # Green for upper band
   RED_COLOR = 'rgb(255, 0, 0)'    # Red for lower band

   # Create the visualization
   fig = make_subplots(specs=[[{"secondary_y": True}]])

   # Add price trace
   fig.add_trace(
       go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], 
                 name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
       secondary_y=False,
   )

   # Add indicator trace
   indicator = merged_df_last_year[indicator_column]
   fig.add_trace(
       go.Scatter(
           x=merged_df_last_year.index,
           y=indicator,
           name=indicator_column,
           line=dict(color=BLUE_COLOR, width=2),
           mode='lines'
       ),
       secondary_y=True,
   )

   # Add band traces if enabled
   if bands == "on":
       # Simply use the provided column names directly
       fig.add_trace(
           go.Scatter(
               x=merged_df_last_year.index,
               y=merged_df_last_year[upper_lower[0]], 
               name="Upper Band",
               line=dict(color=GREEN_COLOR, width=2, dash='dash'),  # Increased width to 2
               mode='lines'
           ),
           secondary_y=True,
       )
       fig.add_trace(
           go.Scatter(
               x=merged_df_last_year.index,
               y=merged_df_last_year[upper_lower[1]], 
               name="Lower Band",
               line=dict(color=RED_COLOR, width=2, dash='dash'),  # Increased width to 2
               mode='lines'
           ),
           secondary_y=True,
       )

   # Add vertical lines for every two months (Jan, Mar, May, Jul, Sep, Nov)
   for month in [1, 4, 7, 10]:
       for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
           date = pd.Timestamp(year=year, month=month, day=1)
           if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
               fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

   # Get the last value of the indicator and format it
   last_value = indicator.iloc[-1]
   
   # Function to format number to K, M, B
   def format_number(num):
       if abs(num) >= 1e9:
           return f"{num/1e9:.2f}B"
       elif abs(num) >= 1e6:
           return f"{num/1e6:.2f}M"
       elif abs(num) >= 1e3:
           return f"{num/1e3:.2f}K"
       else:
           return f"{num:.2f}"
   
   formatted_value = format_number(last_value)

   # Add annotation for the last value
   fig.add_annotation(
       x=0.95,  
       y=last_value*0.88,  
       xref="paper",
       yref="y2", 
       text=formatted_value,
       showarrow=False,
       font=dict(size=18, color=BLUE_COLOR),
       align="left",
       xanchor="left",
       yanchor="middle",
   )

   # Update layout
   fig.update_layout(
       title={
           'text': chart_title,
           'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
       },
       showlegend=False,  
       hovermode="x unified",
       plot_bgcolor='white',
       paper_bgcolor='white',
       font={'color': 'black', 'size': 14},
       width=1000,  
       height=450,  
   )

   # Update axes
   fig.update_xaxes(
       showgrid=False, 
       tickfont={'color': 'black', 'size': 14},
       zeroline=False,
       title_text=''  
   )
   fig.update_yaxes(
       showgrid=False, 
       secondary_y=False, 
       tickfont={'color': GREY_COLOR, 'size': 14},
       zeroline=False,
       showline=True,
       linecolor=GREY_COLOR,
       ticks='outside',
       tickcolor=GREY_COLOR,
       title_text='',
       title_font=dict(size=18)
   )
   fig.update_yaxes(
       showgrid=False, 
       secondary_y=True, 
       tickfont={'color': GREY_COLOR, 'size': 14},
       zeroline=False,
       showline=True,
       linecolor=GREY_COLOR,
       ticks='outside',
       side='right',
       tickcolor=GREY_COLOR,
       title_text='',
       title_font=dict(size=18),
       autorange=True
   )

   return fig

In [20]:
# Create charts for each indicator
indicators = [
    ('rcap_sma_zscore',"Bitcoin: Realized Cap Change", "on", ["rcap_upper", "rcap_lower"]),
    ('sthlth_sma_meandev',"Bitcoin: STH/LTH Supply Ratio", "on", ["sthlth_upper", "sthlth_lower"]),
    ('rhodl_sma_maxmin',"Bitcoin:RHODL Ratio", "on", ["rhodl_upper", "rhodl_lower"]),
    ('rhodlwave_sma_zscore',"Bitcoin: RHODL Wave <3m","on",["rhodlwave_upper","rhodlwave_lower"])
]

for indicator, title,  bands, upper_lower in indicators:
    fig = create_indicator_chart(merged_df, indicator, title, bands, upper_lower)
    
    # Show the plot
    fig.show()
    
    # Optionally, save the plot as an HTML file
    pio.write_html(fig, file=f'bitcoin_analysis_{indicator}_two_year.html')

print("All charts have been displayed and saved as separate HTML files.")

All charts have been displayed and saved as separate HTML files.


In [21]:
merged_df.to_csv('bitcoin_analysis_data_liquidity2.csv', index=True)

In [22]:
merged_df.tail()

,price_usd_close,marketcap_realized_usd,lth_sum,sth_sum,rhodl_ratio,rcap_hodl_waves_24h,rcap_hodl_waves_1d_1w,rcap_hodl_waves_1w_1m,rcap_hodl_waves_1m_3m,rcap_sma_zscore,...,rcap_lower,sthlth_sma_meandev,sthlth_upper,sthlth_lower,rhodl_sma_maxmin,rhodl_upper,rhodl_lower,rhodlwave_sma_zscore,rhodlwave_upper,rhodlwave_lower
t,,,,,,,,,,,,,,,,,,,,,
2025-02-01,100644.206315,8.480443e+11,1.318524e+12,4.008927e+11,4781.095685,0.010806,0.049136,0.116206,0.328669,4.454412,...,0.771174,0.303385,0.265747,0.180042,8.593771,9.438533,8.371311,0.504021,0.455973,0.263251
2025-02-02,97530.620848,8.481240e+11,1.276124e+12,3.873934e+11,4874.548511,0.012298,0.048741,0.113050,0.330314,4.375052,...,0.790052,0.303612,0.266629,0.180115,8.556966,9.438533,8.402402,0.504212,0.457618,0.263974
2025-02-03,101403.090040,8.486629e+11,1.327239e+12,4.060663e+11,5132.884227,0.030166,0.034118,0.111915,0.328088,4.288112,...,0.808896,0.303975,0.267511,0.180202,8.533041,9.438533,8.470122,0.504251,0.459248,0.264713
2025-02-04,97899.163754,8.509200e+11,1.278470e+12,3.915439e+11,5616.989950,0.029634,0.040784,0.112369,0.325110,4.235419,...,0.827962,0.304368,0.268394,0.180306,8.528553,9.438533,8.528553,0.504832,0.460869,0.265467
2025-02-05,96607.113895,8.525494e+11,1.264563e+12,3.916148e+11,6023.504763,0.027720,0.047689,0.106594,0.326266,4.222176,...,0.847338,0.305275,0.269283,0.180426,8.546157,9.438533,8.528553,0.505523,0.462483,0.266235


In [23]:
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Change the grey color to a darker shade
GREY_COLOR = 'rgba(70, 70, 70, 0.9)'  # Darker, more opaque grey

def normalize_value(value, min_val, max_val):
    range_val = max_val - min_val
    if range_val == 0:
        return 0
    normalized = (value - min_val) / range_val * 200 - 100
    return normalized

def format_number(number):
    """Format number with k, M, B suffixes"""
    abs_number = abs(number)
    if abs_number >= 1e9:
        return f"{number/1e9:.2f}B"
    elif abs_number >= 1e6:
        return f"{number/1e6:.2f}M"
    elif abs_number >= 1e3:
        return f"{number/1e3:.2f}k"
    else:
        return f"{number:.2f}"

def create_dot_plot(df, metrics_titles, days=90):
    start_date = df.index[-1] - timedelta(days=days)
    df_filtered = df[df.index >= start_date]
    
    fig = go.Figure()
    y_values = []
    
    for metric, title in metrics_titles.items():
        min_val = df_filtered[metric].min()
        max_val = df_filtered[metric].max()
        current_val = df_filtered[metric].iloc[-1]
        
        # Normalize values
        norm_min = -100
        norm_max = 100
        norm_current = normalize_value(current_val, min_val, max_val)
        
        y_values.append(title)
        
        # Add grey line (darker)
        fig.add_trace(go.Scatter(
            x=[norm_min, norm_max], y=[title, title], mode='lines',
            line=dict(color=GREY_COLOR, width=3),  # Using darker grey
            showlegend=False
        ))
        
        # Add min value (red filled dot)
        fig.add_trace(go.Scatter(
            x=[norm_min], y=[title], mode='markers+text',
            marker=dict(color='red', size=16, symbol='circle'),
            text=[format_number(min_val)],
            textposition="bottom center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Min'
        ))
        
        # Add max value (green filled dot)
        fig.add_trace(go.Scatter(
            x=[norm_max], y=[title], mode='markers+text',
            marker=dict(color='green', size=16, symbol='circle'),
            text=[format_number(max_val)],
            textposition="bottom center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Max'
        ))
        
        # Add current value (grey halo dot)
        fig.add_trace(go.Scatter(
            x=[norm_current], y=[title], mode='markers+text',
            marker=dict(
                color='white',
                size=16,
                line=dict(color=GREY_COLOR, width=3),  # Using darker grey
                symbol='circle'
            ),
            text=[format_number(current_val)],
            textposition="top center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Current'
        ))
    
    # Add vertical dashed lines at -50%, 0%, and 50%
    for value in [-50, 0, 50]:
        fig.add_shape(
            type="line",
            x0=value, x1=value, y0=0, y1=1,
            yref="paper",
            xref="x",
            line=dict(color=GREY_COLOR, width=1, dash="dash")  # Using darker grey
        )
    
    fig.update_layout(
        title={
            'text': f'Metric Values Over Last {days} Days',
            'font': {'color': GREY_COLOR, 'size': 20},  # Using darker grey
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title={
            'text': 'Normalized Value (%)',
            'font': {'color': GREY_COLOR, 'size': 16}  # Using darker grey
        },
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(len(y_values))),
            ticktext=y_values,
            tickfont={'color': GREY_COLOR, 'size': 16},  # Using darker grey
            side='left',
            title_standoff=35
        ),
        height=400 + (len(metrics_titles) * 30),
        width=1200,
        showlegend=False,
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=100, r=100, t=50, b=20),
        font=dict(color=GREY_COLOR, size=14)  # Using darker grey
    )
    
    fig.update_xaxes(
        showgrid=False, 
        range=[-110, 110],
        tickvals=[-100, -50, 0, 50, 100],
        ticktext=['-100%<br>MIN-90D', '-50%', '0%', '50%', '100%<br>MAX-90D'],
        tickfont={'color': GREY_COLOR, 'size': 16},  # Using darker grey
        showline=False,
        zeroline=False
    )
    fig.update_yaxes(showgrid=False)
    
    return fig

In [24]:
# Manually define metrics and their titles
metrics_titles = {

    'rcap_sma_zscore': "Realized Cap Change",
    'sthlth_sma_meandev':"STH/LTH Supply Ratio",
    'rhodlwave_sma_zscore': "RHODL Wave <3M"
    
    # Add more metrics and titles as needed
}

# Create the dot plot
fig = create_dot_plot(merged_df, metrics_titles)

# Show the plot
fig.show()

# Optionally, save the plot as an HTML file
import plotly.io as pio
pio.write_html(fig, file='normalized_liquidity2_raw_metrics_dot_plot.html')